In [1]:
from maelzel.snd import vamptools
from maelzel.snd import audiosample
from maelzel.snd import deverb
import pitchtools as pt
import matplotlib.pyplot as plt
import sndfileio
from math import nan

Checking dependencies
end of score.		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 0.003s, CPU: 0.003s
end of score.		   overall amps:      0.0
	   overall samples out of range:        0
0 errors in performance
Elapsed time at end of performance: real: 0.003s, CPU: 0.003s


In [2]:
samples, sr = sndfileio.sndread("../snd/piano-pitchtest-Cmaj.flac")
samples2 = deverb.removeSustain(samples, sr)

In [3]:
r = vamptools.pyin(samples2, sr, overlap=8, voicedThresholdPercentile=0.1, onsetSensitivity=0.1, threshDistr='beta15', lowAmpSuppressionDb=-70, preciseTime=True, maxRelativeSkew=0.1, rmsPeriod=0.020, minRmsPercentile=0.01)
r

Duplicate times: t0=9.827845805, t1=9.827845805


mnOut size: 1789
m_pitchTrack size: 1789


PyinResult(voicedProbabilityCurve=Linear[0.023219955:10.402539683], smoothPitchCurve=Linear[0.029024944:10.402539683], f0curve=Linear[0.023219955:10.396734694], f0candidates=[(0.023s, [525.3, 261.0, 174.2, 131.6, …], [0.1242, 0.0119, 0.0008, 0.0010, …]), (0.029s, [524.5, 261.5, 174.0, 131.3, …], [0.1391, 0.0118, 0.0027, 0.0023, …]), (0.035s, [528.6, 263.7, 175.8, 131.9, …], [0.1468, 0.0080, 0.0021, 0.0025, …]), (0.041s, [526.0, 262.4, 175.4, 131.5, …], [0.2193, 0.0182, 0.0055, 0.0047, …]), (0.046s, [526.8, 263.3, 175.0, 131.2, …], [0.2278, 0.0290, 0.0149, 0.0102, …]), (0.052s, [528.3, 263.6, 175.7, 131.5, …], [0.1444, 0.0139, 0.0088, 0.0088, …]), (0.058s, [527.7, 263.5, 175.5, 131.5, …], [0.1061, 0.0092, 0.0092, 0.0101, …]), (0.064s, [528.6, 264.2, 175.8, 131.7, …], [0.1022, 0.0087, 0.0087, 0.0101, …]), (0.070s, [529.9, 264.2, 176.0, 131.9, …], [0.0947, 0.0065, 0.0076, 0.0092, …]), (0.075s, [529.0, 264.0, 175.8, 131.7, …], [0.0945, 0.0688, 0.0688, 0.1007, …]), ...]

In [4]:
%matplotlib qt 
import matplotlib.pyplot as plt
axes = audiosample.Sample(samples, sr).plotSpectrogram(maxfreq=6000)
#(r.voicedProbabilityCurve * 6000).plot(axes=axes)
# r.f0curve.plot(axes=axes, linewidth=2, color='#ffff00', show=False)
c = r.smoothPitchCurveNan * (r.numCandidates > 0).nanmask()
c.plot(axes=axes, show=False, n=10000)
# r.smoothPitchCurveNan.plot(axes=axes, linewidth=2, color='#00ffff', show=False, n=10000)
# (r.smoothPitchCurve*2).plot(axes=axes, linewidth=2, color='#ffff00', show=False, n=1000)
# r.f0curve.plot(axes=axes, show=False)
# (r.voicedProbabilityCurve*10000).plot(axes=axes, show=False, n=10000)
# (r.numCandidates * 1000).plot(axes=axes, show=False, n=1000)

QSocketNotifier: Can only be used with threads started with QThread


<Axes: >

In [5]:
parts = [b.aslinear().simplify(threshold=0.1) for b in c[::0.01].split()]
axes = plt.subplot()
for part in parts:
    print(part, len(part.points()[0]))
    part.plot(axes=axes, show=False)
plt.show()


Linear[0.023219955:1.263219955] 6
Linear[1.313219955:2.4232199550000004] 6
Linear[2.4932199550000003:4.873219955] 24
Linear[5.273219955:6.583219955000001] 4
Linear[6.623219955000001:7.123219955000001] 2
Linear[7.513219955:8.063219955000001] 9
Linear[8.313219955000001:8.723219955000001] 7
Linear[8.793219955:9.023219955] 4
Linear[9.093219955:9.333219955] 4
Linear[9.373219955:9.573219954999999] 4
Linear[9.633219955:9.853219955] 5
Linear[9.883219955:10.383219955] 11


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [6]:
import csoundengine as ce
session = ce.Session()




--Csound version 6.19 (double samples) Jan 13 2024
[commit: c53556fbbb69d14554a4a4bd91efa441a1701d70]
libsndfile-1.2.0
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 440.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Pro: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Pro:
SECTION 1:


In [7]:
session.defInstr('vco', r'''
|kamp=0.1, kfreq=1000, iatt=0.005, irel=0.1, isust=0.7|
kgain = kfreq > 100 ? 1 : 0
kgain = lagud:k(kgain, 0.050, 0.100)
aenv = linsegr:a(0, iatt, 1, 0.010, isust, irel, 0)
outch 1, vco2:a(kamp, kfreq) * interp(kgain) * aenv
''')

def schedbpf(bpf, session):
    freqpairs = bpf.flatpairs()
    freqpairs[::2] -= bpf.x0
    synth = session.sched('vco', delay=bpf.x0, dur=bpf.x1 - bpf.x0, kfreq=freqpairs[1]) 
    synth.automate('kfreq', freqpairs, delay=bpf.x0)
    return synth

In [8]:
c = r.smoothPitchCurveNan * (r.voicedProbabilityCurve > 0.01)
c.plot(n=1000)

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


<Axes: >

In [9]:
import bpf4
parts = bpf4.util.split_fragments(r.f0curve)
# parts = split_fragments(r.smoothPitchCurveNan)

with session:
    for part in parts:
        s = schedbpf(part, session=session)

In [13]:
prob = r.voicedProbabilityHistogram.percentileToValue(0.1)
plt.plot((0, r.voicedProbabilityCurve.x1), (prob*10000, prob*10000))
plt.show()

qt.qpa.wayland: Wayland does not support QWindow::requestActivate()
